In [44]:
import json
import numpy as np
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

In [6]:
with open("/Users/ika/Desktop/Fall 2024/CS 7300/assignment_two/data/hetionet-v1.0.json", "r") as f:
    het_json = json.load(f)

In [8]:
het_json['metanode_kinds']

(['Anatomy',
  'Biological Process',
  'Cellular Component',
  'Compound',
  'Disease',
  'Gene',
  'Molecular Function',
  'Pathway',
  'Pharmacologic Class',
  'Side Effect',
  'Symptom'],
 47031)

In [29]:
nodes_df = pd.DataFrame(het_json['nodes'])

nodes_red = nodes_df[nodes_df['kind'].isin(['Side Effect', 'Disease', 'Symptom', 'Compound'])].reset_index(drop=True)
print(nodes_red.shape)

(7861, 4)


In [76]:
nodes_red.head(10)

,kind,identifier,name,data
0,Side Effect,C0023448,Lymphocytic leukaemia,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
1,Side Effect,C0575810,Hands weakness of,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
2,Side Effect,C0025193,Melancholia,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
3,Side Effect,C0578687,Sore eye,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
4,Side Effect,C0919671,Subileus,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
5,Side Effect,C0237849,Skin exfoliation,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
6,Compound,DB00201,Caffeine,"{'license': 'CC BY-NC 4.0', 'source': 'DrugBan..."
7,Side Effect,C0042267,Vaginal inflammation,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
8,Compound,DB09019,Bromhexine,"{'license': 'CC BY-NC 4.0', 'source': 'DrugBan..."
9,Side Effect,C1141885,Sclerosing encapsulating peritonitis,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."


In [69]:
edges_df = pd.DataFrame(het_json['edges'])
edges_df

,source_id,target_id,kind,direction,data
0,"[Anatomy, UBERON:0000178]","[Gene, 9489]",upregulates,both,"{'source': 'Bgee', 'unbiased': True}"
1,"[Anatomy, UBERON:0000992]","[Gene, 9816]",expresses,both,"{'sources': ['Bgee'], 'unbiased': True}"
2,"[Anatomy, UBERON:0002450]","[Gene, 55093]",expresses,both,"{'sources': ['Bgee'], 'unbiased': True}"
3,"[Gene, 801]","[Gene, 7428]",interacts,both,"{'sources': ['II_literature', 'hetio-dag'], 'u..."
4,"[Gene, 5987]","[Gene, 9412]",interacts,both,"{'sources': ['II_literature', 'hetio-dag'], 'u..."
...,...,...,...,...,...
2250192,"[Gene, 120227]","[Biological Process, GO:0044107]",participates,both,"{'source': 'NCBI gene2go', 'license': 'CC BY 4..."
2250193,"[Gene, 10913]","[Biological Process, GO:1902533]",participates,both,"{'source': 'NCBI gene2go', 'license': 'CC BY 4..."
2250194,"[Gene, 51805]","[Pathway, PC7_8621]",participates,both,"{'license': 'CC BY 4.0', 'source': 'Reactome v..."
2250195,"[Compound, DB00555]","[Side Effect, C0032617]",causes,both,"{'source': 'SIDER 4.1', 'license': 'CC BY-NC-S..."


In [70]:
# filtered_egdes = edges_df[edges_df['target_id'].apply(lambda x: any(item in x for item in ['Disease', 'Side Effect', 'Compound, Symptom']))]
# filtered_egdes.shape

In [71]:
edges_df['identifier'] = edges_df['source_id'].apply(lambda x: x[1])
edges_df['source_name'] = edges_df['source_id'].apply(lambda x: x[0])
edges_df['target_name'] = edges_df['target_id'].apply(lambda x: x[0])

In [59]:
nodes_red.head(5)

,kind,identifier,name,data
0,Side Effect,C0023448,Lymphocytic leukaemia,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
1,Side Effect,C0575810,Hands weakness of,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
2,Side Effect,C0025193,Melancholia,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
3,Side Effect,C0578687,Sore eye,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."
4,Side Effect,C0919671,Subileus,"{'source': 'UMLS via SIDER 4.1', 'license': 'C..."


In [72]:
edges_with_source = edges_df.merge(nodes_red, left_on='source_name', right_on='identifier', how='left')
edges_with_source.rename(columns={'kind': 'source_kind'}, inplace=True)

# Merging node information for the target node
edges_with_full_info = edges_with_source.merge(nodes_red, left_on='target_name', right_on='identifier', how='left')
edges_with_full_info.rename(columns={'kind': 'target_kind'}, inplace=True)

# Dropping the extra identifier columns
edges_with_full_info.drop(columns=['identifier_x', 'identifier_y'], inplace=True)

In [73]:
edges_with_full_info.head(5)

,source_id,target_id,kind_x,direction,data_x,source_name,target_name,kind_y,name_x,data_y,target_kind,identifier,name_y,data
0,"[Anatomy, UBERON:0000178]","[Gene, 9489]",upregulates,both,"{'source': 'Bgee', 'unbiased': True}",Anatomy,Gene,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Anatomy, UBERON:0000992]","[Gene, 9816]",expresses,both,"{'sources': ['Bgee'], 'unbiased': True}",Anatomy,Gene,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[Anatomy, UBERON:0002450]","[Gene, 55093]",expresses,both,"{'sources': ['Bgee'], 'unbiased': True}",Anatomy,Gene,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[Gene, 801]","[Gene, 7428]",interacts,both,"{'sources': ['II_literature', 'hetio-dag'], 'u...",Gene,Gene,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[Gene, 5987]","[Gene, 9412]",interacts,both,"{'sources': ['II_literature', 'hetio-dag'], 'u...",Gene,Gene,NaN,NaN,NaN,NaN,NaN,NaN,NaN
